In [166]:
import os
from classesCHANGE import PointCloudGenerator
import time
import plyfile 
import open3d as o3d
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib as plt

In [2]:
#data = plyfile.PlyData.read(r'C:\Users\Jay\Documents\Github\HarvestEye\Avocado100_30deg_group1_densified_point_cloud.ply')
#data.text = True
#data.write('avoFixed.ply')

In [4]:
pcd = o3d.io.read_point_cloud(r'C:\Users\Jay\Documents\Github\HarvestEye\main\avoFixed.ply')
o3d.visualization.draw_geometries([pcd])


In [5]:
# Downsample pcd
voxel_size = 0.2
downsampled_pcd = pcd.voxel_down_sample(voxel_size)
# Convert point cloud to numpy array
points = np.asarray(downsampled_pcd.points)
colors = np.asarray(downsampled_pcd.colors)
# print(colors)

# Concatenate XYZ and RGB values
xyzrgb = np.hstack((points, colors))
scaler = StandardScaler()
xyzrgb_normalized = scaler.fit_transform(xyzrgb)

In [12]:
num_clusters = 2
weights = [5, 5, 500, 1, 550, 1]
weighted_xyzrgb = xyzrgb_normalized*weights[:]
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(weighted_xyzrgb)
k_means_labels = kmeans.labels_
cluster_colors = np.zeros_like(colors)
# Define the colors
green_color = [0, 150/255, 0]  # Green color
brown_color = [0.647, 0.165, 0.165]  # Brown color

# Find the label with the larger cluster
larger_cluster_label = np.argmax(np.bincount(k_means_labels))

# Assign colors based on cluster sizes
for i in range(num_clusters):
    if i == larger_cluster_label:
        cluster_colors[k_means_labels == i] = brown_color # Brown color
    else:
        cluster_colors[k_means_labels == i] = green_color  # Green color

seg_pcd = downsampled_pcd
seg_pcd.colors = o3d.utility.Vector3dVector(cluster_colors)
o3d.visualization.draw_geometries([seg_pcd])

In [14]:
ground_points = points[k_means_labels == larger_cluster_label]
tree_points = points[k_means_labels != larger_cluster_label]

# Assign colors to tree points (green) and ground points (brown)
tree_colors = np.full_like(tree_points[:, :3], [0, 0, 0])
ground_colors = np.full_like(ground_points[:, :3], [1, 1, 1])

# Convert the green points to an Open3D point cloud
tree_pcd = o3d.geometry.PointCloud()
tree_pcd.points = o3d.utility.Vector3dVector(tree_points[:, :3])  # Assuming only xyz data is needed
tree_pcd.colors = o3d.utility.Vector3dVector(tree_colors)

# Visualize the green points (trees)
o3d.visualization.draw_geometries([tree_pcd] + [pcd])

In [8]:
o3d.visualization.draw_geometries([tree_pcd])

In [152]:
#Extract only the xy coordinates
xy_points = tree_points[:, :2]

#Perform DBSCAN clustering on xy coordinates
eps = 0.2  # Adjust this parameter based on the density of your point cloud
min_samples = 1
dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(xy_points)
dbscan_labels = dbscan.labels_

#Now, retrieve the Z coordinates for clustered tree points
clustered_tree_indices = np.where(dbscan_labels != -1)[0]  # Indices of points that are not noise
#Now, retrieve the Z coordinates for clustered tree points
clustered_tree_points = tree_points[clustered_tree_indices]

import random
#Convert clustered_tree_points to an Open3D point cloud
dbscan_clusters_pcd = o3d.geometry.PointCloud()
dbscan_clusters_pcd.points = o3d.utility.Vector3dVector(clustered_tree_points)

# Assign random colors to each cluster
cluster_colors = {}
for label in np.unique(dbscan_labels):
    if label == -1:  # Skip noise points
        continue
    color = [random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1)]  # Generate a random RGB color
    cluster_colors[label] = color

# Assign colors to each point based on its cluster
point_colors = []
for label in dbscan_labels:
    if label == -1:  # Noise points
        point_colors.append([0.5, 0.5, 0.5])  # Gray color
    else:
        point_colors.append(cluster_colors[label])

# Set colors for the clustered tree points
dbscan_clusters_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Set the visualization mode to "unlit" for precise color rendering
o3d.visualization.draw_geometries([dbscan_clusters_pcd])# + [pcd])

In [167]:
min_cluster_size = 5  # Minimum number of points for a cluster to be considered valid
max_cluster_size = 1000  # Maximum number of points for a cluster to be considered valid
dbscan_labels_filtered = dbscan_labels[dbscan_labels != -1] # Remove the noise label (-1)
ls, cs = np.unique(dbscan_labels_filtered, return_counts=True)
dic = dict(zip(ls,cs))

# Create a list idx that stores each index that matches the required max and min cluster sizes
idx = [] 

for i, label in enumerate(dbscan_labels_filtered):
    if dic[label] >= min_cluster_size and label <= max_cluster_size:
        idx.append(i)

# Convert the point cloud to a numpy array
points_array = np.asarray(dbscan_clusters_pcd.points)

# Filter the points using the valid indices
filtered_points = points_array[idx]

# Assign deterministic colors to each cluster
num_clusters = len(np.unique(dbscan_labels_filtered))
colors = plt.cm.tab10(np.arange(num_clusters))[:, :3]  # Use a colormap for deterministic colors

# Assign colors to each point based on its cluster
point_colors = []
for label in dbscan_labels_filtered:
        color = colors[label % num_clusters]  # Use deterministic colors
        point_colors.append(color)

# Create a new point cloud from the filtered points
filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
filtered_pcd.colors = o3d.utility.Vector3dVector(point_colors)

# Visualize the filtered point cloud
o3d.visualization.draw_geometries([filtered_pcd])